In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from trl import SFTTrainer, SFTConfig

In [ ]:
# model_name = "data"
model_name = "data"
dataset_name = "data"
# device_map = 'auto'
device_map = {"": 0}

In [ ]:
peft_config = LoraConfig(
      lora_alpha=16,
      lora_dropout=0.1,
      r=64,
      bias="none",
      task_type="CAUSAL_LM",
)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

In [ ]:
def prompt_instruction_format(sample):
  return f"""### Instruction:
    Use the Task below and the Input given to write the Response, which is a programming code that can solve the following Task:

    ### Task:
    {sample['instruction']}

    ### Input:
    {sample['input']}

    ### Response:
    {sample['output']}
    """

In [ ]:
code_dataset = load_dataset(dataset_name, split='train')

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name,
          quantization_config=bnb_config,
          use_cache = False,
          device_map=device_map)
model.config.pretraining_tp = 1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
training_args = SFTConfig(
    output_dir="data",
    num_train_epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    weight_decay=0.001,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=False,
    lr_scheduler_type="cosine",
    disable_tqdm=True,
    seed=42,
    max_seq_length=2048,
    packing=True,
    report_to="none",
    neftune_noise_alpha=5
    )

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=code_dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,
    formatting_func=prompt_instruction_format,
    args=training_args,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('data')

In [ ]:
trained_model = AutoModelForCausalLM.from_pretrained('data')
trained_tokenizer = AutoTokenizer.from_pretrained('data')

In [ ]:
# instruction="Write a Python program to generate a binary search  given an array input and element to be searched."
# input="[2,3,4,5], [5]"

instruction="Write a Python program which calculates Fibonacci numbers with given number in input."
input="25"

prompt = f"""### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

### Task:
{instruction}

### Input:
{input}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids

In [ ]:
print(f"-------------------------\n\n")
print(f"Prompt:\n{prompt}\n")
print(f"-------------------------\n\n")

outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=200, do_sample=True, top_p=0.9,temperature=0.5)

print(f"-------------------------\n\n")
print(f"Generated instruction:\n{trained_tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n\n")